# TensorFlores - Example 3

## 1. Importing libraries

In [ ]:
import sys, os
path = os.getcwd().split('\\examples')[0]
sys.path.append(path)

In [ ]:
#!pip install seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflores.utils.clustering import ClusteringMethods
from tensorflores.utils.quantization import Quantization
from tensorflores.utils.cpp_generation import CppGeneration
from tensorflores.utils.json_handle import JsonHandle

## 2.  Create the TensorFlores class

## 3. Load model from json

In [ ]:
model_as_json = JsonHandle().load_json_model('./tensorflores_without_quant')
model_as_json

## 4. Post-Training Quantization

### 4.1 Evolving post-training quantization

#### 4.1.1 Choose the quantization method

- Autocloud 
    - bias_clustering_method = Clustering_method.autocloud_biases(threshold_biases = 1.4148)
    - weight_clustering_method = Clustering_method.autocloud_weight(threshold_weights = 1.4148)

- Meanshift
    - weight_clustering_method = Clustering_method.meanshift_weight(bandwidth_weights = 0.005)
    - bias_clustering_method = Clustering_method.meanshift_biases(bandwidth_biases = 0.005)

- Affinity Propagation
    - weight_clustering_method = Clustering_method.affinity_propagation_weight(affinityprop_damping_bias = 0.65)
    - bias_clustering_method = Clustering_method.affinity_propagation_biases(affinityprop_damping_weight = 0.7)

- DBSTREAM
    - weight_clustering_method = Clustering_method.dbstream_weight(clustering_threshold_bias = 0.8)
    - bias_clustering_method = Clustering_method.dbstream_biases(clustering_threshold_weight = 0.1)

In [ ]:
Clustering_method = ClusteringMethods()

bias_clustering_method = Clustering_method.autocloud_biases(threshold_biases = 1.4)
weight_clustering_method = Clustering_method.autocloud_weight(threshold_weights = 1.4)

#### 4.1.2 Define the metric to calculate the distance between the centroids of the clusters and the values of weights and/or biases

- "euclidean"
- "manhattan"
- "minkowski"
- "chebyshev"
- "cosine"
- "hamming"
- "bray_curtis"
- "jaccard"
- "wasserstein"
- "dtw"

In [ ]:
model_as_json_quant = Quantization().post_training_quantization(json_data = model_as_json,
                                        quantization_type = 'evolving', 
                                        distance_metric = "dtw", 
                                        bias_clustering_method = bias_clustering_method,
                                        weight_clustering_method = weight_clustering_method)

In [ ]:
model_as_json_quant

#### 4.1.3 Save model as C++

In [ ]:
cpp_model = CppGeneration().generate_cpp_from_json(json_data = model_as_json_quant, file_name = 'tensorflores_evolving_PQT')
cpp_model

### 4.2 Int8 post-training quantization

#### 4.2.1 Apply int8 quantizaion

In [ ]:
model_as_json_int8 = Quantization().post_training_quantization(json_data = model_as_json,
                                        quantization_type = 'int8')

#### 4.2.2 Save model as C++

In [ ]:
model_as_json_int8

In [ ]:
cpp_model_int8 = CppGeneration().generate_cpp_from_json(json_data = model_as_json_int8, file_name = 'tensorflores_int8_PQT')

In [ ]:
df = pd.read_csv('./result/output_PQT_int8.csv', sep=';')

In [ ]:
error = df['Predicted'].values - df['Real'].values 

In [ ]:
df['Time (us)'].values[1:].mean()

In [ ]:
# Estatísticas
error_mean = np.mean(error)
error_std = np.std(error)
error_max = np.max(error)
error_min = np.min(error)

# Criando a figura e os eixos
fig, ax = plt.subplots(figsize=(8, 6))

# Estilo do gráfico
plt.style.use('ggplot')

# Plotando histograma com KDE (Densidade) sem legenda
sns.histplot(error, bins=30, color='coral', edgecolor='black', stat="density", kde=True, legend=False, ax=ax)

# Configuração dos eixos
ax.set_xlabel('Error', fontsize=14)
ax.set_ylabel('Density', fontsize=14)
#ax.set_title('Error Distribution', fontsize=15)
ax.tick_params(labelsize=14)
ax.grid(True)

# Definindo limites do eixo X
#ax.set_xlim(error_min, error_max)

# Exibir os valores estatísticos no gráfico
textstr = '\n'.join((
    f'Max: {error_max:.4f}',
    f'Min: {error_min:.4f}',
    f'Mean: {error_mean:.4f}',
    f'2x Std: {2 * error_std:.4f}'
))
props = dict(boxstyle='round', facecolor='gray', alpha=0.5)
ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=12,
        verticalalignment='top', horizontalalignment='right', bbox=props)

plt.savefig('./figures/example_04_float32.pdf', bbox_inches='tight', pad_inches=0, dpi=300)
# Exibir gráfico
plt.show()

In [ ]:
df = pd.read_csv('./result/output_PQT_evolving.csv', sep=';')

In [ ]:
error = df['Predicted'].values - df['Real'].values 

In [ ]:
# Estatísticas
error_mean = np.mean(error)
error_std = np.std(error)
error_max = np.max(error)
error_min = np.min(error)

# Criando a figura e os eixos
fig, ax = plt.subplots(figsize=(8, 6))

# Estilo do gráfico
plt.style.use('ggplot')

# Plotando histograma com KDE (Densidade) sem legenda
sns.histplot(error, bins=30, color='coral', edgecolor='black', stat="density", kde=True, legend=False, ax=ax)

# Configuração dos eixos
ax.set_xlabel('Error', fontsize=14)
ax.set_ylabel('Density', fontsize=14)
#ax.set_title('Error Distribution', fontsize=15)
ax.tick_params(labelsize=14)
ax.grid(True)

# Definindo limites do eixo X
#ax.set_xlim(error_min, error_max)

# Exibir os valores estatísticos no gráfico
textstr = '\n'.join((
    f'Max: {error_max:.4f}',
    f'Min: {error_min:.4f}',
    f'Mean: {error_mean:.4f}',
    f'2x Std: {2 * error_std:.4f}'
))
props = dict(boxstyle='round', facecolor='gray', alpha=0.5)
ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=12,
        verticalalignment='top', horizontalalignment='right', bbox=props)

plt.savefig('./figures/example_03_evolving.pdf', bbox_inches='tight', pad_inches=0, dpi=300)
# Exibir gráfico
plt.show()